Ensure no data leaking by replacing post game stats with previous averaging

In [ ]:


df.read_csv("../raw/schedules_raw.csv", index=False)
print(df.head())

NameError: name 'df' is not defined

Calculate metrics using scheduling data